<a href="https://colab.research.google.com/github/Prof-Awing/Mobile-User-Behavior-Classifier/blob/main/mobile_user_bhv_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## THIS CELL IS TO BE USED IN GOOGLE COLAB
'''
!kaggle datasets download valakhorasani/mobile-device-usage-and-user-behavior-dataset
!unzip mobile-device-usage-and-user-behavior-dataset.zip
!rm -rf mobile-device-usage-and-user-behavior-dataset.zip
!ls
'''

Dataset URL: https://www.kaggle.com/datasets/valakhorasani/mobile-device-usage-and-user-behavior-dataset
License(s): apache-2.0
  0% 0.00/11.3k [00:00<?, ?B/s]
100% 11.3k/11.3k [00:00<00:00, 17.9MB/s]
Archive:  mobile-device-usage-and-user-behavior-dataset.zip
  inflating: user_behavior_dataset.csv  
mobile-device-usage-and-user-behavior-dataset.zip  sample_data	user_behavior_dataset.csv


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [6]:
df= pd.read_csv('user_behavior_dataset.csv')
df.head()

,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1,Google Pixel 5,Android,393,6.4,1872,67,1122,40,Male,4
1,2,OnePlus 9,Android,268,4.7,1331,42,944,47,Female,3
2,3,Xiaomi Mi 11,Android,154,4.0,761,32,322,42,Male,2
3,4,Google Pixel 5,Android,239,4.8,1676,56,871,20,Male,3
4,5,iPhone 12,iOS,187,4.3,1367,58,988,31,Female,3


In [7]:
label_encoders = {}
for column in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

df.head()


,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1,0,0,393,6.4,1872,67,1122,40,1,4
1,2,1,0,268,4.7,1331,42,944,47,0,3
2,3,3,0,154,4.0,761,32,322,42,1,2
3,4,0,0,239,4.8,1676,56,871,20,1,3
4,5,4,1,187,4.3,1367,58,988,31,0,3


In [10]:
X = df.drop(['User Behavior Class'], axis=1)
y = df['User Behavior Class']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into train, cv, test sets and leave a portion for 'new'
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled[:560], y[:560], test_size=0.3, random_state=42)
X_cv, X_test, y_cv, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Use a separate portion of the original data for 'new'
X_new = X_scaled[560:600]
y_new = y[560:600]

# Print the shapes of the datasets
print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)
print(y_train.shape)
print(y_cv.shape)
print(y_test.shape)
print(X_new.shape)
print(y_new.shape)

(392, 10)
(84, 10)
(84, 10)
(392,)
(84,)
(84,)
(40, 10)
(40,)


In [11]:
model= Sequential([
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(6, activation='linear'),
])
model.compile(optimizer=Adam(learning_rate=0.001), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_data=(X_cv, y_cv))

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.1058 - loss: 1.8053 - val_accuracy: 0.2976 - val_loss: 1.6710
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3523 - loss: 1.5666 - val_accuracy: 0.3571 - val_loss: 1.5370
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4370 - loss: 1.4644 - val_accuracy: 0.4048 - val_loss: 1.4134
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5741 - loss: 1.3076 - val_accuracy: 0.4167 - val_loss: 1.2998
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5932 - loss: 1.1735 - val_accuracy: 0.5000 - val_loss: 1.1856
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5767 - loss: 1.0923 - val_accuracy: 0.5595 - val_loss: 1.0691
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6655 - loss: 0.9671 - val_accuracy: 0.6429 - val_loss: 0.9706
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7439 - loss: 0.8319 - val_accuracy: 0.7381 - 

In [12]:
import numpy as np
from scipy.special import softmax  # Import softmax from scipy

y_pred = model.predict(X_test)
y_pred = softmax(y_pred, axis=1)  # Use softmax from scipy
y_pred = np.argmax(y_pred, axis=1)
y_pred

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([2, 1, 1, 1, 1, 4, 4, 2, 2, 5, 2, 3, 5, 3, 5, 1, 1, 3, 5, 1, 2, 1,
       1, 5, 2, 5, 3, 2, 3, 1, 3, 3, 1, 5, 1, 5, 3, 2, 5, 3, 2, 4, 4, 5,
       2, 1, 1, 3, 1, 3, 5, 1, 4, 4, 4, 3, 2, 3, 3, 1, 1, 3, 3, 5, 2, 2,
       3, 3, 5, 2, 3, 1, 5, 4, 5, 2, 3, 4, 5, 2, 2, 3, 3, 1])

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print("accuracy on test:", accuracy_score(y_test, y_pred))

[[20  0  0  0  0]
 [ 0 17  0  0  0]
 [ 0  0 22  0  0]
 [ 0  0  0  9  0]
 [ 0  0  0  0 16]]
accuracy on test: 1.0


In [17]:
y_hat= model.predict(X_new)
y_hat= softmax(y_hat, axis=1)
y_hat= np.argmax(y_hat, axis=1)
print(confusion_matrix(y_new, y_hat))
print("accuracy on new data:",accuracy_score(y_new, y_hat))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[[7 0 0 0 0]
 [0 9 0 0 0]
 [0 0 8 0 0]
 [0 0 0 9 0]
 [0 0 0 0 7]]
accuracy on new data: 1.0
